# **Sentiment Analysis of Drug Reviews using LLMs**

### Notebook Overview: Sentiment Analysis and Text Summarization Experiments with LLM APIs

This notebook explores experiments conducted using various models and APIs for sentiment analysis and text summarization tasks. Key highlights include:

- **BERT Sentiment Analysis:**
- Conducted sentiment analysis using a `BERT`-based model.
  - Optimized processing through batch processing on MPS (Metal Performance Shaders) `GPU` , significantly reducing inference time.

- **Sentiment Analysis with LLM APIs:**
  - Leveraged `Gemini API`, `OpenAI API`, and `GROQ API` for sentiment analysis.
  - Optimized Gemini API by analyzing sentiments for 100 drug reviews in a single batch call, improving efficiency and overcoming free-tier limitations.

- **Review Summarization:** 
  - Used Gemini API to generate concise summaries of drug reviews, providing aggregated insights for each drug.

These experiments aim to evaluate the performance and scalability of different models and APIs for text-based healthcare analytics, demonstrating practical optimizations for handling large datasets effectively.


In [23]:
import pandas as pd 

df =  pd.read_csv("preprocessed_reviews.csv")
df.head(5)

,drugName,condition,review,rating,date,usefulCount,prep_review,prep_review_token,prep_review_stpwrd
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,2012-05-20,27,it have no side effect I take it in combinatio...,"['it', 'have', 'no', 'side', 'effect', 'I', 't...",effect combination bystolic mg fish oil
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,2010-04-27,192,my son be halfway through his fourth week of i...,"['my', 'son', 'be', 'halfway', 'through', 'his...",son halfway fourth week intuniv concerned begi...
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,2009-12-14,17,I use to take another oral contraceptive which...,"['I', 'use', 'to', 'take', 'another', 'oral', ...",oral contraceptive pill cycle happy light peri...
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,2015-11-03,10,this be my first time use any form of birth co...,"['this', 'be', 'my', 'first', 'time', 'use', '...",time form birth control m glad go patch month ...
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,2016-11-27,37,suboxone have completely turn my life around I...,"['suboxone', 'have', 'completely', 'turn', 'my...",suboxone completely turn life feel healthy m e...


In [24]:
df

,drugName,condition,review,rating,date,usefulCount,prep_review,prep_review_token,prep_review_stpwrd
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,2012-05-20,27,it have no side effect I take it in combinatio...,"['it', 'have', 'no', 'side', 'effect', 'I', 't...",effect combination bystolic mg fish oil
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,2010-04-27,192,my son be halfway through his fourth week of i...,"['my', 'son', 'be', 'halfway', 'through', 'his...",son halfway fourth week intuniv concerned begi...
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,2009-12-14,17,I use to take another oral contraceptive which...,"['I', 'use', 'to', 'take', 'another', 'oral', ...",oral contraceptive pill cycle happy light peri...
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,2015-11-03,10,this be my first time use any form of birth co...,"['this', 'be', 'my', 'first', 'time', 'use', '...",time form birth control m glad go patch month ...
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,2016-11-27,37,suboxone have completely turn my life around I...,"['suboxone', 'have', 'completely', 'turn', 'my...",suboxone completely turn life feel healthy m e...
...,...,...,...,...,...,...,...,...,...
161292,Campral,Alcohol Dependence,"""I wrote my first report in Mid-October of 201...",10,2015-05-31,125,I write my first report in midoctober of I hav...,"['I', 'write', 'my', 'first', 'report', 'in', ...",write report midoctober alcohol post reduce do...
161293,Metoclopramide,Nausea/Vomiting,"""I was given this in IV before surgey. I immed...",1,2011-11-01,34,I be give this in iv before surgey I immediate...,"['I', 'be', 'give', 'this', 'in', 'iv', 'befor...",give iv surgey immediately anxious sit pa say ...
161294,Orencia,Rheumatoid Arthritis,"""Limited improvement after 4 months, developed...",2,2014-03-15,35,limit improvement after month develop bad rash...,"['limit', 'improvement', 'after', 'month', 'de...",limit improvement month develop bad rash md re...
161295,Thyroid desiccated,Underactive Thyroid,"""I&#039;ve been on thyroid medication 49 years...",10,2015-09-19,79,I ve be on thyroid medication year I spend my ...,"['I', 've', 'be', 'on', 'thyroid', 'medication...",ve thyroid medication year spend synthroid t4 ...


#### Initial experiment of sentiment analysis using TextBlob


In [446]:
# Just a very basic approach to sentiment analysis
from textblob import TextBlob

# Define the sentiment classification function
def classify_sentiment(polarity):
    if polarity > 0.1:  # Adjust threshold as needed
        return 'pos'
    elif polarity < -0.1:
        return 'neg'
    else:
        return 'neu'
    
df['polarity'] = df['review'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Apply the classification function
df['sentiment_textBlob'] = df['polarity'].apply(classify_sentiment)

In [ ]:
pd.set_option('display.max_colwidth', None)
df[[df.columns[0], df.columns[1], 'review', 'sentiment']].head(10)

,drugName,condition,review,sentiment
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil""",neu
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. \r\nWe have tried many different medications and so far this is the most effective.""",pos
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, which had 21 pill cycle, and was very happy- very light periods, max 5 days, no other side effects. But it contained hormone gestodene, which is not available in US, so I switched to Lybrel, because the ingredients are similar. When my other pills ended, I started Lybrel immediately, on my first day of period, as the instructions said. And the period lasted for two weeks. When taking the second pack- same two weeks. And now, with third pack things got even worse- my third period lasted for two weeks and now it&#039;s the end of the third week- I still have daily brown discharge.\r\nThe positive side is that I didn&#039;t have any other side effects. The idea of being period free was so tempting... Alas.""",neu
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth control. I&#039;m glad I went with the patch, I have been on it for 8 months. At first It decreased my libido but that subsided. The only downside is that it made my periods longer (5-6 days to be exact) I used to only have periods for 3-4 days max also made my cramps intense for the first two days of my period, I never had cramps before using birth control. Other than that in happy with the patch""",pos
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around. I feel healthier, I&#039;m excelling at my job and I always have money in my pocket and my savings account. I had none of those before Suboxone and spent years abusing oxycontin. My paycheck was already spent by the time I got it and I started resorting to scheming and stealing to fund my addiction. All that is history. If you&#039;re ready to stop, there&#039;s a good chance that suboxone will put you on the path of great life again. I have found the side-effects to be minimal compared to oxycontin. I&#039;m actually sleeping better. Slight constipation is about it for me. It truly is amazing. The cost pales in comparison to what I spent on oxycontin.""",pos
5,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard erections however experianced headache, lower bowel preassure. 3rd day erections would wake me up &amp; hurt! Leg/ankles aches severe lower bowel preassure like you need to go #2 but can&#039;t! Enjoyed the initial rockhard erections but not at these side effects or $230 for months supply! I&#039;m 50 &amp; work out 3Xs a week. Not worth side effects!""",neu
6,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I took the Plan B 26 hours later, and took a pregnancy test two weeks later - - I&#039;m pregnant.""",pos
7,Aripiprazole,Bipolar Disorde,"""Abilify changed my life. There is hope. I was on Zoloft and Clonidine when I first started Abilify at the age of 15.. Zoloft for depression and Clondine to manage my complete rage. My moods were out of control. I was depressed and hopeless one second and then mean, irrational, and full of rage the next. My Dr. prescribed me 2mg of Abilify and from that

----

### BERT sentiment analysis

In [ ]:
import torch
print(torch.__version__)

2.4.0


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

# # Load pre-trained model and tokenizer
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Understanding Padding and Truncation in BERT Tokenization

- **Padding**:
  - Padding is applied to ensure that all sequences within a batch have the same length.
  - The tokenizer pads sequences to the length of the longest sequence in the current batch.
  - This ensures uniformity in input dimensions for batch processing.

- **Truncation**:
  - Truncation is applied to sequences that exceed a specified maximum length (usually 512 tokens for BERT).
  - It ensures that sequences longer than the model’s maximum allowable length are cut down to fit within the limit.
  - Truncation is crucial to prevent errors when sequences exceed the model’s input capacity.

- **Combined Use**:
  - Padding and truncation are often used together:
    - **Padding** makes sure all sequences in a batch are of equal length.
    - **Truncation** ensures that none of the sequences exceed the model's maximum length.

**Example**: For BERT, sequences are padded to the longest sequence in the batch, but any sequence longer than 512 tokens is truncated to fit within the model's limit.


In [ ]:
# # Tokenize input
# inputs = tokenizer("The product was a complete waste of money.", return_tensors="pt")

# # Make prediction
# outputs = model(**inputs)
# prediction = torch.argmax(outputs.logits).item()
# Below we can use the tokenization as the part of the pipeline rather than doing it manually.

##### First performed without using batching or GPU (no optimization, more time consuming)

In [ ]:
from transformers import pipeline


# Initialize the sentiment analysis pipeline with custom tokenizer parameters
sentiment_analysis = pipeline(
    'sentiment-analysis', 
    model="distilbert-base-uncased-finetuned-sst-2-english",
    tokenizer="distilbert-base-uncased-finetuned-sst-2-english",
    padding=True,              # Enable padding
    truncation=True,           # Enable truncation
    max_length=512             # Set maximum sequence length
)


# # Apply the sentiment analysis pipeline to each review in the DataFrame
# df['sentiment'],df['score'] = df['review'].apply(lambda x: pd.Series(sentiment_analysis(x)[0]['label']))



In [ ]:
df[['sentiment', 'score']] = df['review'].apply(lambda x: pd.Series(sentiment_analysis(x)[0]))
# very much time consuming, need to optimize (64 minutes)

In [ ]:
df[['drugName', 'review', 'sentiment', 'score']][15:20]

,drugName,review,sentiment,score
15,Liraglutide,"""I have been taking Saxenda since July 2016. I had severe nausea for about a month once I got up to the 2.6 dosage. It has since subsided and the only side effect I notice now is the dry mouth. I make sure to drink 2.5 litres of water a day (about 10 glasses). This helps with the weight loss as well as the constipation. I have been reducing my dose to find a comfortable spot where I am still losing weight but don&#039;t feel like I am over medicating. For me, 1.8 is working very well. I also feel wearing a Fitbit has really helped. I can track my food, water, exercise and steps - it keeps me moving more. When this started I could barely walk the length of myself without getting winded - I have lost 58 lbs so far.""",POSITIVE,0.659131
16,Trimethoprim,"""This drug worked very well for me and cleared up my UTI in a matter of 48hrs, although I was on a 7 day course of 2x200mg/daily. Unfortunately once the tablets finished the infection returned so needed a 2nd course. I&#039;m currently taking a low dose at evening to keep the infection at bay while the specialists do some tests. I normally get every side effect going and dread taking new medication in any form especially after an horrific experience with Ciorofloxacin but the only side effect I experienced with this was itchy ankles and feet on day two but that didn&#039;t last into third day so I would class this as a minor side effect and insignificant. Good luck.""",NEGATIVE,0.998607
17,Amitriptyline,"""I&#039;ve been taking amitriptyline since January 2013 after being diagnosed with fibromyalgia. I tried cymbalta for a week and my Dr decided that since it kept me awake 24/7 that it wasn&#039;t a good fit for me even though it relieved my pain. I started with 25 mg. After 2 weeks I had some breakthrough pain, so my Dr increased my dosage to 50mg and I&#039;m still on that dosage today. For the first full year I was on this I never had pain and felt &#039;cured&#039;. As time went on, it seems to help control my pain. I do still have pain, but at a low manageable level. I take this around 7pm each evening to avoid feeling overly drowsy in the mornings. This med allows me to have a fairly normal life.""",POSITIVE,0.500642
18,Lamotrigine,"""I&#039;ve been on every medicine under the sun (it seems) to manage the hypomania / mania of Bipolar2. Within a few days of starting Lamictal (was on Tegretol) it was as if someone had turned a light on in my head. Is this the way &quot;normal&quot; people think? was one of the first things that struck me. I sleep like a log, have no irritability, almost no anger (other than what would be considered normal). I am able to focus and my creative, impulsive thoughts have decreased enough to be manageable. It&#039;s a God send as I would perennially be in a state of hypo mania, which sounds great, but I was physically and emotionally exhausted. My libido is markedly lower but I wonder if that is perhaps an effect of not being hypo manic. I take 20 mg for anxiety.""",NEGATIVE,0.994631
19,Nilotinib,"""I have been on Tasigna for just over 3 years now (300mg x 2 times a day) Tasigna worked for me within a few weeks I have been in remission for basically the entire 3 years. As for the side effects, I feel sick to my stomach, tired, severe leg and arm pain, I can&#039;t walk to long without needing a break, I went from working full time to not working at all because I am so tired all the time.... but hey I am in remission and alive so is all the above really a problem? Nope""",NEGATIVE,0.968858


In [ ]:
from datasets import Dataset
import pandas as pd


# Convert your pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)
# Huigging face dataset functionality - more optimized and efficient data storage using Apache Arrow, allows us to work with large datasets which may not fit in memory

In [ ]:
dataset

Dataset({
    features: ['drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'prep_review', 'prep_review_token', 'prep_review_stpwrd'],
    num_rows: 161297
})

### Understanding Batch Processing in BERT

**Batch Size and Input Tensor:**
- **Batch Size (`N`)**: The number of sequences (e.g., reviews) processed simultaneously.
- **Sequence Length (`L`)**: The number of tokens in each sequence (up to 512 for BERT).
- **Hidden Size (`H`)**: The dimensionality of the embeddings and hidden states inside BERT (usually 768 for BERT-base).

When processing a batch of 256 reviews with BERT, the input tensor typically has the shape `[N, L, H] = [256, 512, 768]`.

**Parallel Processing in BERT:**
- **Vectorized Operations**: BERT performs matrix multiplications and other operations on all elements of the batch in parallel. Modern hardware, especially GPUs, is optimized for these large tensor operations.
- **Layer-by-Layer Processing**: BERT processes the entire batch layer by layer, where each layer's operations are applied to all sequences in the batch simultaneously.
- **Attention Mechanism**: BERT's attention mechanisms compute relationships between tokens in a highly parallelized manner, leveraging the batch structure to maximize computational efficiency.

**Efficiency Considerations:**
- **Memory Usage**: Larger batch sizes require more memory, as the model needs to store intermediate activations for all sequences in the batch.
- **Computation**: While the computational load increases with batch size, parallel processing on GPUs significantly reduces the time taken compared to sequential processing.

**Output Tensor:**
- After processing, BERT outputs a tensor with shape `[N, L, H]`. For tasks like sentiment analysis, this might be reduced to `[N, num_labels]` (e.g., `[256, 2]` for binary classification), where each sequence in the batch receives a probability distribution over the possible labels.

**Key Takeaway:**
- **Batch processing** allows BERT to efficiently handle multiple sequences in parallel, making full use of modern hardware capabilities. The batch size is not limited by BERT itself but by the available memory and computational power of the hardware.

This understanding helps in optimizing the processing of large datasets with BERT, balancing batch size with available resources to maximize efficiency.


In [ ]:
import torch

In [ ]:
# Check if MPS (Metal Performance Shaders) is available (GPU Check for MacBook)
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS (Metal) device found!")
else:
    device = torch.device("cpu")
    print("MPS device not found. Using CPU.")

MPS (Metal) device found!


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from datasets import Dataset
from tqdm import tqdm
import pandas as pd


# Load the model and tokenizer
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

# Check for MPS and move model to MPS
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
model.to(device)

# Initialize the sentiment analysis pipeline
# sentiment_analysis = pipeline(
#     'sentiment-analysis', 
#     model=model,
#     tokenizer=tokenizer,
#     device=device
# )
sentiment_analysis = pipeline(
    'sentiment-analysis', 
    model="distilbert-base-uncased-finetuned-sst-2-english",
    tokenizer="distilbert-base-uncased-finetuned-sst-2-english",
    padding=True,
    truncation=True,
    max_length=512, # token length allowed by BERT model
    batch_size=256,  # Process reviews in batches of 32
    device=0 if device == torch.device("mps") else -1  # Set device=0 for MPS, -1 for CPU

)


# Convert your pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# # Tokenize function (was trying to tokenize seperately but the tokenizer in pipeline streamlines this process automatically according to the model in use)
# def tokenize_function(examples):
#     return tokenizer(examples["review"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")

# # Tokenize the dataset
# tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=1)  # num_proc=1 for M1/M2/M3 Macs



In [ ]:
# Define the function to apply the sentiment analysis in batches
def compute_sentiment(batch):
    with torch.no_grad(): # disable gradient computation for inference/evaluation
        results = sentiment_analysis(batch['review'])
    return {'sentiment': [result['label'] for result in results]}

# Apply the function to the dataset
processed_dataset = dataset.map(compute_sentiment, batched=True, batch_size=32, desc="Processing")

# Analysis time reduced by 50%

Processing:   0%|          | 0/161297 [00:00<?, ? examples/s]

In [ ]:
# Convert the results back to a pandas DataFrame
results_df = processed_dataset.to_pandas()
print("Sentiment analysis complete!")
results_df.head()


Sentiment analysis complete!


,drugName,condition,review,rating,date,usefulCount,prep_review,prep_review_token,prep_review_stpwrd,sentiment
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,2012-05-20,27,it have no side effect I take it in combinatio...,"['it', 'have', 'no', 'side', 'effect', 'I', 't...",effect combination bystolic mg fish oil,NEGATIVE
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,2010-04-27,192,my son be halfway through his fourth week of i...,"['my', 'son', 'be', 'halfway', 'through', 'his...",son halfway fourth week intuniv concerned begi...,POSITIVE
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,2009-12-14,17,I use to take another oral contraceptive which...,"['I', 'use', 'to', 'take', 'another', 'oral', ...",oral contraceptive pill cycle happy light peri...,NEGATIVE
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,2015-11-03,10,this be my first time use any form of birth co...,"['this', 'be', 'my', 'first', 'time', 'use', '...",time form birth control m glad go patch month ...,NEGATIVE
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,2016-11-27,37,suboxone have completely turn my life around I...,"['suboxone', 'have', 'completely', 'turn', 'my...",suboxone completely turn life feel healthy m e...,POSITIVE


In [ ]:
results_df[['drugName','sentiment']]

,drugName,sentiment
0,Valsartan,NEGATIVE
1,Guanfacine,POSITIVE
2,Lybrel,NEGATIVE
3,Ortho Evra,NEGATIVE
4,Buprenorphine / naloxone,POSITIVE
...,...,...
161292,Campral,NEGATIVE
161293,Metoclopramide,NEGATIVE
161294,Orencia,NEGATIVE
161295,Thyroid desiccated,POSITIVE


### Comparison of performance between TextBlob and Hugging Face BERT

In [ ]:
# For making the column output similar to the Bert output
df.loc[df["sentiment_textBlob"] == "pos", "sentiment_textBlob"] = "POSITIVE"


In [ ]:
df.loc[df["sentiment_textBlob"] == "neg", "sentiment_textBlob"] = "NEGATIVE"
df.loc[df["sentiment_textBlob"] == "neu", "sentiment_textBlob"] = "NEUTRAL"

In [ ]:
results_df['sentiment'].eq(df['sentiment_textBlob']).value_counts()

False    104560
True      56737
Name: count, dtype: int64

In [ ]:
results_df['sentiment'].eq(df['sentiment_textBlob']).value_counts(normalize=True) * 100

False    64.824516
True     35.175484
Name: proportion, dtype: float64

35% values of the senimtment column are equal to the sentiment_textBlob where 64% are not equal. Thus, there is distinctive difference between the two models sentiment output.

###  Sentiment Analysis

In [ ]:
# Majority sentiment in the dataset
results_df['sentiment'].value_counts(normalize=True)*100

sentiment
NEGATIVE    67.970266
POSITIVE    32.029734
Name: proportion, dtype: float64

In [ ]:
results_df[['drugName','sentiment']].value_counts(normalize=True)#.mul(100).unstack() #.fillna(0).reset_index()

drugName                           sentiment
Levonorgestrel                     NEGATIVE     0.016138
Etonogestrel                       NEGATIVE     0.015419
Ethinyl estradiol / norethindrone  NEGATIVE     0.013174
Nexplanon                          NEGATIVE     0.010285
Ethinyl estradiol / norgestimate   NEGATIVE     0.009430
                                                  ...   
Grifulvin V                        NEGATIVE     0.000006
Gris-PEG                           POSITIVE     0.000006
Guaifenesin / hydrocodone          NEGATIVE     0.000006
                                   POSITIVE     0.000006
A + D Cracked Skin Relief          NEGATIVE     0.000006
Name: proportion, Length: 5691, dtype: float64

In [ ]:
(results_df.groupby("drugName")['sentiment']
 .value_counts(normalize=True)
#  .mul(100) # Multiply by 100 to convert proportions to percentages
#  .unstack()
#  .reset_index().fillna(0)
 )

drugName                              sentiment
A + D Cracked Skin Relief             NEGATIVE     1.000000
A / B Otic                            POSITIVE     1.000000
Abacavir / dolutegravir / lamivudine  NEGATIVE     0.769231
                                      POSITIVE     0.230769
Abacavir / lamivudine / zidovudine    POSITIVE     1.000000
                                                     ...   
ZzzQuil                               NEGATIVE     1.000000
depo-subQ provera 104                 NEGATIVE     1.000000
ella                                  NEGATIVE     0.784314
                                      POSITIVE     0.215686
femhrt                                NEGATIVE     1.000000
Name: proportion, Length: 5691, dtype: float64

In [ ]:
mod_df  = results_df.groupby('drugName')['sentiment'].count().rename('review_count').reset_index()
# .value_counts(normalize=True).mul(100).reset_index()

,drugName,review_count
0,A + D Cracked Skin Relief,1
1,A / B Otic,1
2,Abacavir / dolutegravir / lamivudine,52
3,Abacavir / lamivudine / zidovudine,1
4,Abatacept,19
...,...,...
3431,Zyvox,17
3432,ZzzQuil,2
3433,depo-subQ provera 104,1
3434,ella,51


In [ ]:
top_drugs = results_df["drugName"].value_counts().reset_index()
sorted_top_drugs = top_drugs.sort_values(by =  "count", ascending=False)

In [ ]:
sorted_top_drugs

,drugName,count
0,Levonorgestrel,3657
1,Etonogestrel,3336
2,Ethinyl estradiol / norethindrone,2850
3,Nexplanon,2156
4,Ethinyl estradiol / norgestimate,2117
...,...,...
2901,Follicle stimulating hormone,1
2902,Dazidox,1
2903,Calcium carbonate / risedronate,1
2904,Aquavite-E,1


In [ ]:
drugs_sentiment = sorted_top_drugs['drugName'].loc[sorted_top_drugs['count'] > 10]

In [ ]:
sentiment_df = results_df.loc[results_df['drugName'].isin(drugs_sentiment)]

Before

In [ ]:
(sentiment_df.groupby("drugName")['sentiment']
 .value_counts(normalize=True)
 .mul(100) # Multiply by 100 to convert proportions to percentages
 .reset_index()
)

,drugName,sentiment,proportion
0,Abacavir / dolutegravir / lamivudine,NEGATIVE,76.923077
1,Abacavir / dolutegravir / lamivudine,POSITIVE,23.076923
2,Abatacept,POSITIVE,57.894737
3,Abatacept,NEGATIVE,42.105263
4,Abilify,NEGATIVE,63.228700
...,...,...,...
2635,Zyrtec,POSITIVE,39.130435
2636,Zyvox,NEGATIVE,82.352941
2637,Zyvox,POSITIVE,17.647059
2638,ella,NEGATIVE,78.431373


After unstack

In [ ]:
sntmt_count = (sentiment_df.groupby("drugName")['sentiment']
 .value_counts(normalize=True)
 .mul(100)
 .unstack()
 .reset_index()
)

sntmt_count = sntmt_count.rename_axis(None, axis=1)
sntmt_count
# unstack changes the level of a MultiIndex (rows) into columns, 
# effectively pivoting the 'sentiment' index level to columns.
# This means that for each drugName, we'll now have separate columns 
# for each sentiment value (e.g., POSITIVE, NEGATIVE) with their corresponding proportions.
# Now its easy to analyze the senitment for each drugName.

,drugName,NEGATIVE,POSITIVE
0,Abacavir / dolutegravir / lamivudine,76.923077,23.076923
1,Abatacept,42.105263,57.894737
2,Abilify,63.228700,36.771300
3,Abreva,77.848101,22.151899
4,Acamprosate,50.458716,49.541284
...,...,...,...
1317,Zyprexa,66.467066,33.532934
1318,Zyprexa Zydis,50.000000,50.000000
1319,Zyrtec,60.869565,39.130435
1320,Zyvox,82.352941,17.647059


In [ ]:
print('Average positive sentiment across dataset: ',sntmt_count['POSITIVE'].mean())
print('Average negative sentiment across dataset: ',sntmt_count['NEGATIVE'].mean())

Average positive sentiment across dataset:  32.36548212180383
Average negative sentiment across dataset:  67.73244671971447


The majority of sentiment in the dataset is negative.

In [ ]:
fig = px.bar(sntmt_count.sort_values(by='POSITIVE', ascending=False), x='drugName', y='POSITIVE'
             )
fig.show()

In [ ]:
fig = px.bar(sntmt_count.sort_values(by='NEGATIVE', ascending=False), x='drugName', y='NEGATIVE')
fig.show()

In [ ]:
sntmt_count.loc[sntmt_count['POSITIVE'] > 50] # using where clause produces NAN when condition is not met,
# sntmt_count.query('POSITIVE > 50') #another way


,drugName,NEGATIVE,POSITIVE
1,Abatacept,42.105263,57.894737
17,Acetaminophen / phenyltoloxamine,27.272727,72.727273
21,Acidophilus,38.461538,61.538462
33,Adderall,43.283582,56.716418
34,Adderall XR,45.614035,54.385965
...,...,...,...
1236,Ventolin,45.454545,54.545455
1264,Warfarin,44.444444,55.555556
1269,Xanax,46.929825,53.070175
1270,Xanax XR,39.473684,60.526316


**List of drugs having Majority of Negative Sentiment (> 50% each):**

In [ ]:
sntmt_count.query('NEGATIVE > 50')
#net promoter score

,drugName,NEGATIVE,POSITIVE
0,Abacavir / dolutegravir / lamivudine,76.923077,23.076923
2,Abilify,63.228700,36.771300
3,Abreva,77.848101,22.151899
4,Acamprosate,50.458716,49.541284
5,Acanya,51.111111,48.888889
...,...,...,...
1316,Zyclara,76.923077,23.076923
1317,Zyprexa,66.467066,33.532934
1319,Zyrtec,60.869565,39.130435
1320,Zyvox,82.352941,17.647059


**List of drugs having equal proportion of positive and negative sentiment (50% each):**

In [ ]:
sntmt_count.query('POSITIVE == 50 and NEGATIVE == 50')


,drugName,NEGATIVE,POSITIVE
52,Aleve,50.0,50.0
82,Amoxil,50.0,50.0
115,Ativan,50.0,50.0
134,Axert,50.0,50.0
161,Benzaclin,50.0,50.0
165,Benzoyl peroxide / erythromycin,50.0,50.0
278,Clorazepate,50.0,50.0
291,Commit,50.0,50.0
309,Cyanocobalamin,50.0,50.0
325,Demerol,50.0,50.0


In [ ]:
results_df['sentiment'].value_counts(normalize=False)*100

sentiment
NEGATIVE    10963400
POSITIVE     5166300
Name: count, dtype: int64

### **LLM based Sentiment Analysis** 

Experimenting with Google Gemini API

In [ ]:
import google.generativeai as genai
import pandas as pd

# Configure the Gemini API with your API key
genai.configure(api_key=GOOGLE_API_KEY)

# Define the Gemini model and settings
defaults = {
    'model': 'models/text-bison-001',
    'temperature': 0.5, 
    'candidate_count': 1,
    'top_k': 40,
    'top_p': 0.95,
    'max_output_tokens': 1024,
}

# Define a function to get the sentiment of a review using Gemini API
def get_sentiment(review):
    prompt = f"""Tell me whether the following sentence's sentiment is positive, negative, or neutral.
    Sentence: {review}
    Sentiment: """
    
    # Send the prompt to the Gemini API
    response = genai.generate_text(
        **defaults,
        prompt=prompt
    )
    
    # Extract the sentiment result
    return response.result



Test case

In [ ]:
# Sample DataFrame with 'review' column
test_df = pd.DataFrame({
    'review': ["I love this product!", "This is terrible.", "I have mixed feelings about this."]
})

# Apply the sentiment function to the 'review' column
test_df['sentiment'] = test_df['review'].apply(get_sentiment)

# Display the DataFrame with the sentiment results
print(test_df)

                              review sentiment
0               I love this product!  positive
1                  This is terrible.  negative
2  I have mixed feelings about this.   neutral


Utilizing on dataset

In [ ]:
t_df  = df.copy()


In [ ]:
t_df["sentiment_gemini"] = t_df["review"][:3].apply(get_sentiment)

AttributeError: 'NoneType' object has no attribute 'from_call'

In [ ]:
t_df["sentiment_gemini"][:3]

0        None
1    positive
2        None
Name: sentiment_gemini, dtype: object

Retry with Gemini API

**Testing with Gemini API**

In [ ]:
import pandas as pd
# Dummy DataFrame
g_df = pd.DataFrame({'review': ['This movie was amazing!', 'I hated this book.', 'Not bad, but not great.']})

Using Gemini API: system instruction to steer behaviour of model. It initializes the model with your preferred context for the prompt-repsonse behaviour. "Product-level behavior can be specified here, separate from prompts provided by end users." - Google Docs

In [ ]:
model=genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  system_instruction="You are customer review sentiment analyzer. You reply with positive, negative or neutral.") #system instructions to steer behaviour of model

# You are customer review sentiment analyzer for medical drugs review. You reply with positive, negative or neutral.
# Define a function to get the sentiment of a review using Gemini API
def get_sentiment(review):
    
    # Send the prompt to the Gemini API
    response = model.generate_content(review)
    
    # Extract the sentiment result
    return response.text

# Apply the function to the 'review' column
g_df['sentiment'] = g_df['review'].apply(get_sentiment)

# Print the updated DataFrame
print(g_df)

                    review    sentiment
0  This movie was amazing!  Positive \n
1       I hated this book.  Negative \n
2  Not bad, but not great.   Neutral \n


The above results depict the accurate performance of the testing with Gemini API

Now further testing on dataset

In [ ]:
# Checking out copied dataframe
t_df[:5]

,drugName,condition,review,rating,date,usefulCount,prep_review,prep_review_token,prep_review_stpwrd
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,2012-05-20,27,it have no side effect I take it in combinatio...,"['it', 'have', 'no', 'side', 'effect', 'I', 't...",effect combination bystolic mg fish oil
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,2010-04-27,192,my son be halfway through his fourth week of i...,"['my', 'son', 'be', 'halfway', 'through', 'his...",son halfway fourth week intuniv concerned begi...
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,2009-12-14,17,I use to take another oral contraceptive which...,"['I', 'use', 'to', 'take', 'another', 'oral', ...",oral contraceptive pill cycle happy light peri...
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,2015-11-03,10,this be my first time use any form of birth co...,"['this', 'be', 'my', 'first', 'time', 'use', '...",time form birth control m glad go patch month ...
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,2016-11-27,37,suboxone have completely turn my life around I...,"['suboxone', 'have', 'completely', 'turn', 'my...",suboxone completely turn life feel healthy m e...


In [ ]:
model=genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  system_instruction="You are customer review sentiment analyzer for medical drugs review. You reply only with Positve, Negative or Neutral.") #system instructions to steer behaviour of model

# import enum
# Define a function to get the sentiment of a review using Gemini API
def get_sentiment(review):
    
    # Send the prompt to the Gemini API
    response = model.generate_content(review,
                                     safety_settings=[
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"} #safety settings turned off as the review does have sex-related content.
],

                                      generation_config=genai.GenerationConfig(
        response_mime_type="text/x.enum",
        response_schema={
            "type": "STRING",
            "enum": ["Positve", "Negative", "Neutral"], # using enum for structured output as google docs
           }
        )
    )

    
    # Extract the sentiment result
    return response.text

Lets first test the model on 10 reviews of the dataset

In [ ]:
# Apply the function to the 'review' column
t_df['sentiment'] = t_df['review'][:].apply(get_sentiment)

# Print the updated DataFrame
t_df[:10]

,drugName,condition,review,rating,date,usefulCount,prep_review,prep_review_token,prep_review_stpwrd,sentiment
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,2012-05-20,27,it have no side effect I take it in combinatio...,"['it', 'have', 'no', 'side', 'effect', 'I', 't...",effect combination bystolic mg fish oil,Neutral
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,2010-04-27,192,my son be halfway through his fourth week of i...,"['my', 'son', 'be', 'halfway', 'through', 'his...",son halfway fourth week intuniv concerned begi...,Neutral
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,2009-12-14,17,I use to take another oral contraceptive which...,"['I', 'use', 'to', 'take', 'another', 'oral', ...",oral contraceptive pill cycle happy light peri...,Negative
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,2015-11-03,10,this be my first time use any form of birth co...,"['this', 'be', 'my', 'first', 'time', 'use', '...",time form birth control m glad go patch month ...,Neutral
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,2016-11-27,37,suboxone have completely turn my life around I...,"['suboxone', 'have', 'completely', 'turn', 'my...",suboxone completely turn life feel healthy m e...,Positve
5,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,2015-11-28,43,2nd day on 5 mg start to work with rock hard e...,"['2nd', 'day', 'on', '5', 'mg', 'start', 'to',...",2nd day 5 mg start work rock hard erection exp...,Negative
6,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",1,2017-03-07,5,he pull out but he cumme a bit in I I take the...,"['he', 'pull', 'out', 'but', 'he', 'cumme', 'a...",pull cumme bit take plan b hour later take pre...,Negative
7,Aripiprazole,Bipolar Disorde,"""Abilify changed my life. There is hope. I was...",10,2015-03-14,32,abilify change my life there be hope I be on z...,"['abilify', 'change', 'my', 'life', 'there', '...",abilify change life hope zoloft clonidine star...,Positve
8,Keppra,Epilepsy,""" I Ve had nothing but problems with the Kepp...",1,2016-08-09,11,I ve have nothing but problem with the keppera...,"['I', 've', 'have', 'nothing', 'but', 'problem...",ve problem keppera constant shake arm amp leg ...,Negative
9,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8,2016-12-08,1,I have be on the pill for many year when my do...,"['I', 'have', 'be', 'on', 'the', 'pill', 'for'...",pill year doctor change rx chateal effective h...,Negative


In [ ]:
t_df["sentiment"].iloc[5]

'Negative'

In [ ]:
t_df.shape

(161297, 10)

Now applying the model on large number of rows testing the api call limit

In [ ]:
# Apply the function to the 'review' column
t_df['sentiment'] = t_df['review'][:500].apply(get_sentiment)

# Print the updated DataFrame
t_df[:500] # can't call api error message

Getting an *error* as the `API call free version` is `limited` in the `Requests per Minute`. But the experiment on the small scale of data showcases promising capabilities of the model/experimentation.

#### Text Summarization Experiment

Now lets conduct experiment on summarising the reviews for each drug using gemini.


As we know the usage limitation Gemini API, we will experiment on a smaller scale of data, i,e, using 1 or 2 drugNames.

In [ ]:
s_df = df.copy() # sample dataframe for summarization experiment

In [ ]:
# drug_counts = s_df["drugName"].value_counts()

# # Use boolean indexing with map to filter out drug names with a count of 100 or more
# s_df[s_df["drugName"].map(drug_counts) ==100]
# s_df.groupby("drugName").filter(lambda x: len(x) == 100)

In [ ]:
# # Get the value counts for each drugName
# drug_counts = s_df["drugName"].value_counts()

# # Filter drug names based on their occurrence being less than 100
# filtered_drug_names = drug_counts[drug_counts == 100].index

In [ ]:
# s_df.loc[ 
s_df[["drugName", 'condition']].value_counts()[s_df[["drugName", 'condition']].value_counts() < 100] #] #[["drugName", "condition"]].

# Lets select Paxil and depression for analysis

drugName                         condition                
Effexor XR                       Anxiety                      99
Bystolic                         High Blood Pressure          99
Euflexxa                         Osteoarthritis               98
Paxil                            Depression                   98
Monistat 3-Day Combination Pack  Vaginal Yeast Infection      98
                                                              ..
Keytruda                         Melanoma                      1
                                 Non-Small Cell Lung Cance     1
Kineret                          Schnitzler Syndrome           1
Kionex                           Hyperkalemia                  1
femhrt                           Postmenopausal Symptoms       1
Name: count, Length: 8176, dtype: int64

In [ ]:
# filter_df = s_df.groupby("drugName").filter(lambda x: len(x) == 100)

In [ ]:
# filter_df = filter_df.drop(columns=['date', 'prep_review', 'prep_review_token', 'prep_review_stpwrd'])

In [ ]:
# n_drug = filter_df.loc[filter_df['drugName'].isin(['Naloxegol'])] # or filter_df[filter_df['drugName'] == 'Naloxegol']
# n_drug

In [ ]:
p_drug = s_df[(s_df['drugName'] == 'Paxil') & (s_df['condition'] == 'Depression')]

p_drug = p_drug.drop(columns=['date', 'prep_review', 'prep_review_token', 'prep_review_stpwrd'])
p_drug

,drugName,condition,review,rating,usefulCount
1236,Paxil,Depression,"""Paxil does help with depression, anxiety and ...",8,13
1656,Paxil,Depression,"""Significant weight gain experienced within a ...",4,26
2655,Paxil,Depression,"""I have taken Paxil for about 6 months. I have...",8,27
6357,Paxil,Depression,"""Paxil saved my life. I was depressed and cou...",8,56
10721,Paxil,Depression,"""My depression is weird, mostly due to taperin...",7,14
...,...,...,...,...,...
153434,Paxil,Depression,"""The good:\r\n\r\n- Improved motivation\r\n- M...",5,17
155431,Paxil,Depression,"""I&#039;ve been on Paxil for two months now an...",8,40
156530,Paxil,Depression,"""Took this med for a month at 20mg and I was o...",10,65
160239,Paxil,Depression,"""When I started Paxil 7 months ago I was clini...",10,108


In [ ]:
p_drug['rating'].agg(["mean", "median"])


mean      6.938776
median    8.000000
Name: rating, dtype: float64

In [ ]:
review_text =' '.join(p_drug['review'])

In [ ]:
review_text

'"Paxil does help with depression, anxiety and obsessive compulsive disorder. It makes you lose weight but the weight comes back fast when you stop taking it." "Significant weight gain experienced within a few months.  VERY tough on the libido. Extremely difficult to get off this medication without significant side effects - dizziness, nausea. I was taking Celexa before and would definitely go back. For the purposes of banishing depression/anxiety, it worked, but not without side effects that make me want to stop taking it each day." "I have taken Paxil for about 6 months. I have not had any weight gain as I had previously with other medications.  I feel emotionally much more stable and the side effects have been minimal.  I still feel tired occasionally and have some trouble with my short term-memory but my overall quality of life seems to be better and although I still feel mildly blue at times the feeling is not as intense and doesn&#039;t linger for weeks.\r\n\r\nHappy with the med

In [ ]:
model=genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  system_instruction="You are customer review summariser for medical drugs review. You will provide a concise objective summary about customer overall experience from the corpus of customer reviews.") #system instructions to steer behaviour of model

# import enum
# Define a function to get the sentiment of a review using Gemini API
def get_summary(review):
    
    # Send the prompt to the Gemini API
    response = model.generate_content(review,
                                     safety_settings=[
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"} #safety settings turned off as the review does have sex-related content.
]
# ,

#                                       generation_config=genai.GenerationConfig(
#         response_mime_type="text/x.enum",
#         response_schema={
#             "type": "STRING",
#             "enum": ["Positve", "Negative", "Neutral"], # using enum for structured output as google docs
#            }
#         )
    )

    
    # Extract the sentiment result
    return response.text

In [ ]:
summary  = get_summary(review_text)

In [ ]:
print(summary)

Paxil is generally effective in treating depression, anxiety and obsessive-compulsive disorder. Many users reported significant improvement in their mood and anxiety levels after taking Paxil. However, the medication can cause side effects such as weight gain, decreased libido, and sleepiness. Some users also reported experiencing withdrawal symptoms when trying to stop taking Paxil. It is important to consult with a doctor before taking Paxil, as it may not be suitable for everyone.



In [ ]:
import pprint

pprint.pprint(summary)

('Paxil is generally effective in treating depression, anxiety and '
 'obsessive-compulsive disorder. Many users reported significant improvement '
 'in their mood and anxiety levels after taking Paxil. However, the medication '
 'can cause side effects such as weight gain, decreased libido, and '
 'sleepiness. Some users also reported experiencing withdrawal symptoms when '
 'trying to stop taking Paxil. It is important to consult with a doctor before '
 'taking Paxil, as it may not be suitable for everyone.\n')


Text Summarization Results: (For Paxil and Depression)

*Paxil is generally effective in treating depression, anxiety and obsessive-compulsive disorder. Many users reported significant improvement in their mood and anxiety levels after taking Paxil. However, the medication can cause side effects such as weight gain, decreased libido, and sleepiness. Some users also reported experiencing withdrawal symptoms when trying to stop taking Paxil. It is important to consult with a doctor before taking Paxil, as it may not be suitable for everyone.*

### **Lets try Batch processing:**

In [ ]:
def gen_func(itera,n):
    m = len(itera)
    for i in range(0, m, n):
        yield itera[i:i+n]


In [ ]:
l = [i for i in range(51)]
for i in gen_func(l, 8):
    print(i)

[0, 1, 2, 3, 4, 5, 6, 7]
[8, 9, 10, 11, 12, 13, 14, 15]
[16, 17, 18, 19, 20, 21, 22, 23]
[24, 25, 26, 27, 28, 29, 30, 31]
[32, 33, 34, 35, 36, 37, 38, 39]
[40, 41, 42, 43, 44, 45, 46, 47]
[48, 49, 50]


In [ ]:
b_df = df.iloc[:1000,:4].copy() # doubt on why to use copy, as copy creates a different object and any change within the 
# two wont be reflected in other. Rest if not then both are
# basically same objects and get affected by change of value in any one. This is because you're working with a view, not a copy.
b_df

,drugName,condition,review,rating
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9
...,...,...,...,...
995,Duac,Acne,"""The product is working so well for my athleti...",8
996,Liothyronine,Underactive Thyroid,"""as a supplement to levothyroxine it has been ...",10
997,Dextromethorphan,Cough,"""This worked great for my husband until he gav...",1
998,Dapsone,Acne,"""I have tried almost everything under the sun ...",10


---

##### Series of experiment for batch processing:

In [ ]:
model=genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  system_instruction="You are customer review sentiment analyzer for medical drugs reviews. You reply only with Positve, Negative or Neutral. Analyze the sentiment of each of the following drug reviews in the list seperated by /n") #system instructions to steer behaviour of model

# import enum
# Define a function to get the sentiment of a review using Gemini API
def get_sentiment(review):
    
    # Send the prompt to the Gemini API
    response = model.generate_content(review,
                                     safety_settings=[
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"} #safety settings turned off as the review does have sex-related content.
],

        generation_config=genai.GenerationConfig(
            response_mime_type="application/json",
            response_schema={
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "review_number": {"type": "integer"},
                        "sentiment": {
                            "type": "string",
                            "enum": ["Positive", "Negative", "Neutral"]
                        }
                    },
                    "required": ["review_number", "sentiment"]
                }
            }
        )
    )

    
    # Extract the sentiment result
    # return response.text
    result = json.loads(response.text)
    return {item['review_number']: item['sentiment'] for item in result}

In [375]:
test_review = b_df['review'][:100].to_list()
# get_sentiment(test_review )

In [415]:
import json
# import genai

# Initialize the GenerativeModel with updated system instructions
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction=(
        "You are a customer review sentiment analyzer for medical drugs reviews. "
        "You will receive multiple reviews given in a list. "
        "For each review in the list, respond with its sentiment: Positive, Negative, or Neutral. "
        "Return the sentiments as a JSON array in the same order as the reviews."
    )
)

# Define a function to get the sentiment of multiple reviews using Gemini API in a single batch
def get_batch_sentiments(reviews):
    # Format the reviews as a single string separated by line breaks
    # batch_prompt = "\n\n".join(reviews)
    
    # Send the batch prompt to the Gemini API
    response = model.generate_content(
        reviews,
        safety_settings=[
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"}  # Safety settings turned off as the review does have sex-related content.
        ],
        generation_config=genai.GenerationConfig(
            response_mime_type="application/json",  # Expecting JSON array response with sentiments
            response_schema={
                "type": "ARRAY",  # Indicates we're expecting an array of results
                "items": {
                    "type": "STRING",
                    "enum": ["Positive", "Negative", "Neutral"]  # Using enum for structured output
                }
            }
        )
    )
    
    # Debugging: Print the raw response
    print("Raw response from API:", response.text)
    
    # Extract and return the list of sentiments
    try:
        sentiments = json.loads(response.text)  # Parsing JSON response
        if not isinstance(sentiments, list):
            raise ValueError("The API response is not a list.")
        return sentiments
    except json.JSONDecodeError as e:
        print(f"JSON decoding failed: {e}")
        return []
    except ValueError as ve:
        print(f"Unexpected response format: {ve}")
        return []

# # Example list of reviews for batch processing
# test_reviews = [
#     "This drug helped me a lot with my condition, no side effects experienced.",
#     "Terrible experience! The medication made my symptoms worse.",
#     "No significant change after taking the drug for a month.",
#     # Add more reviews as needed
# ]

# Get the batch sentiments
batch_sentiments = get_batch_sentiments(test_review[40:50])

# # Print the results
# for idx, (review, sentiment) in enumerate(zip(test_review, batch_sentiments), 1):
#     print(f"Review {idx}: {review}\nSentiment: {sentiment}\n")

# Debugging: Check lengths
print(f"Length of input reviews: {len(test_review)}")
print(f"Length of output sentiments: {len(batch_sentiments)}")


Raw response from API: ["Positive","Positive","Positive","Negative","Negative","Positive","Positive","Positive","Negative","Positive"] 
Length of input reviews: 100
Length of output sentiments: 10


**Results:**
The above experiment results in unreliable results for batch processing, as we can observe the difference in the input and output of sentiments. (100 inputs we receive 10 output sentiments, we need to strategize to enforce expected consistent and reliable output). 

**Exp:** Now changing the input as a list of reviews seperated by /n making it specific for the model to interpret each review to output respective sentiment. Also using enforced JSON output format parameter to restrict the output.

In [411]:
# Initialize the GenerativeModel with the desired model and system instructions
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction="You are a customer review sentiment analyzer for medical drugs review. You reply only with Positive, Negative, or Neutral."
)

# Define a function to get the sentiment of multiple reviews using Gemini API in a single batch
def get_batch_sentiments(reviews):
    # Combine all reviews into a single prompt, separated by new lines or another delimiter if required
    batch_prompt = "\n".join(reviews)  # Assuming each review is treated separately within the prompt
    
    # Send the batch prompt to the Gemini API
    response = model.generate_content(
        batch_prompt,
        safety_settings=[
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"}  # Safety settings turned off as the review does have sex-related content.
        ],
        generation_config=genai.GenerationConfig(
            response_mime_type="application/json",  # Expecting JSON array response with sentiments
            response_schema={
                "type": "ARRAY",  # Indicates we're expecting an array of results
                "items": {
                    "type": "STRING",
                    "enum": ["Positive", "Negative", "Neutral"]  # Using enum for structured output
                }
            }
        )
    )
    
    # Extract and return the list of sentiments
    # Assuming response.text contains a JSON array of sentiments
    sentiments = json.loads(response.text)  # Parsing JSON response
    return sentiments

# List of reviews for batch processing
# t_reviews = [
#     "This drug helped me a lot with my condition, no side effects experienced.",
#     "Terrible experience! The medication made my symptoms worse.",
#     "No significant change after taking the drug for a month."
# ]

# Get the batch sentiments
batch_sentiments = get_batch_sentiments(test_review)

# # Print the results
# for review, sentiment in zip(test_review, batch_sentiments):
#     print(f"Review: {review}\nSentiment: {sentiment}\n")


In [385]:
print(len(test_review))
print(len(batch_sentiments))    # print(r)

100
78


**Results:**

The above experiment still showcases the misinterpretation of the input by the model in batch processing failing to generate the required output sentiments.
We need to strategize to enforce output in a more formatted manner. 

**Exp:** Now we use a strategy using separators ('\n') and local indexing for each review. Ensuring the API correctly interprets large batches of reviews.
- This approach ensures accurate interpretation of input by the API and consistent output.

In [427]:
import pandas as pd
# import genai
from tqdm import tqdm

# Assuming you've already set up the Gemini model as before
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction=("You are a customer review sentiment analyzer for medical drug reviews. "
                        "Analyze the sentiment of each review and classify it as Positive, Negative, or Neutral. "
                        "Provide your response as a JSON array where each element contains a review number and its sentiment.")
)

def get_sentiments_batch(reviews, batch_size=100):
    all_sentiments = {}
    
    for i in tqdm(range(0, len(reviews), batch_size)):
        batch = reviews[i:i+batch_size]
        formatted_reviews = "\n".join([f"{j+1}. {review}" for j, review in enumerate(batch)])
        
        prompt = f"Analyze the sentiment of each of the following drug reviews:\n\n{formatted_reviews}"

        response = model.generate_content(
            prompt,
            safety_settings=[
                {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"}
            ],
            generation_config=genai.GenerationConfig(
                response_mime_type="application/json",
                response_schema={
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "review_number": {"type": "integer"},
                            "sentiment": {
                                "type": "string",
                                "enum": ["Positive", "Negative", "Neutral"]
                            }
                        },
                        "required": ["review_number", "sentiment"]
                    }
                }
            )
        )

        try:
            result = json.loads(response.text)
            for item in result:
                all_sentiments[i + item['review_number'] - 1] = item['sentiment']
        except json.JSONDecodeError:
            print(f"Error parsing JSON response for batch starting at index {i}. Raw response:")
            print(response.text)

    return all_sentiments

# Assuming your original dataframe is called 'df' and has a 'review' column
# reviews = df['review'].tolist()

# Get sentiments for all reviews
sentiments = get_sentiments_batch(test_review)
print(len(sentiments))
# Create a new 'sentiment' column in the original dataframe
# df['sentiment'] = df.index.map(sentiments)

# Display the updated dataframe
# print(df)

100%|██████████| 1/1 [00:05<00:00,  5.37s/it]

100


**This optimization improved the reliability and accuracy of sentiment analysis results. As now llm is able to take into account each review passed in the batch with the help of local indexing, and return a sentiment associated with the index value of the review, thus fixing the issue of missing reviews/sentiments and unreliable results.**

---

**Successful Experiment:**

For 1000 rows dataframe

In [447]:
import pandas as pd
import json
from tqdm import tqdm


model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction=(
        "You are a customer review sentiment analyzer for medical drug reviews. "
        "Analyze the sentiment of each review and classify it as Positive, Negative, or Neutral. "
        "Provide your response as a JSON array where each element contains a review number and its sentiment."
    )
)

def get_sentiments_batch(reviews, batch_size=100):
    all_sentiments = {}
    
    for i in tqdm(range(0, len(reviews), batch_size)):
        batch = reviews[i:i+batch_size]
        
        # Formatting reviews with numbering for batch prompt
        formatted_reviews = "\n".join([f"{j + 1}. {review}" for j, review in enumerate(batch)]) #local indexing will input 1. review, 2.review so same will be the output, thus
        prompt = f"Analyze the sentiment of each of the following drug reviews:\n\n{formatted_reviews}" # we need to adjust idx further after output to match the mainlist.

        # Send batch prompt to Gemini API
        response = model.generate_content(
            prompt,
            safety_settings=[
                {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"}
            ],
            generation_config=genai.GenerationConfig(
                response_mime_type="application/json",
                response_schema={
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "review_number": {"type": "integer"},
                            "sentiment": {
                                "type": "string",
                                "enum": ["Positive", "Negative", "Neutral"]
                            }
                        },
                        "required": ["review_number", "sentiment"]
                    }
                }
            )
        )

        try:
            # Parse the JSON response
            result = json.loads(response.text)
            
            # Ensure result is a list and iterate over it
            # Calculate the absolute index in the main list of reviews.
# The variable 'i' represents the starting index of the current batch in the overall list of reviews.
# 'item['review_number']' is the local index of the review within the batch, starting from 1.
# We subtract 1 to adjust for 0-based indexing, since the main list of reviews is indexed from 0.
# Therefore, the formula 'i + item['review_number'] - 1' converts the local index of the review in the batch
# to the corresponding index in the main list, ensuring we correctly map each review sentiment back to its
# original position in the full list of reviews.
            if isinstance(result, list):
                for item in result:
                    # Calculate the absolute index in all_sentiments
                    absolute_index = i + item['review_number'] - 1
                    all_sentiments[absolute_index] = item['sentiment']
            else:
                print(f"Unexpected response format for batch starting at index {i}.")
                print(f"Raw response: {response.text}")
        
        except json.JSONDecodeError:
            print(f"Error parsing JSON response for batch starting at index {i}. Raw response:")
            print(response.text)
        except KeyError:
            print(f"Unexpected keys in response for batch starting at index {i}. Expected keys: 'review_number' and 'sentiment'.")
            print(f"Raw response: {response.text}")

    return all_sentiments

# Assuming your original dataframe is called 'df' and has a 'review' column
reviews = b_df['review'].tolist()

# Example: Get sentiments for all reviews
sentiments = get_sentiments_batch(b_df['review'].to_list())
print(f"Number of processed sentiments: {len(sentiments)}")
# Create a new 'sentiment' column in the original dataframe
b_df['sentiment'] = b_df.index.map(sentiments)

# For debugging, print out any missing reviews
missing_reviews = [idx for idx in range(len(test_review)) if idx not in sentiments]
if missing_reviews:
    print(f"Missing sentiments for the following review indices: {missing_reviews}")


100%|██████████| 10/10 [00:53<00:00,  5.36s/it]

Number of processed sentiments: 1000


In [449]:
b_df

,drugName,condition,review,rating,sentiment
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,Neutral
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,Positive
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,Negative
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,Neutral
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,Positive
...,...,...,...,...,...
995,Duac,Acne,"""The product is working so well for my athleti...",8,Positive
996,Liothyronine,Underactive Thyroid,"""as a supplement to levothyroxine it has been ...",10,Positive
997,Dextromethorphan,Cough,"""This worked great for my husband until he gav...",1,Negative
998,Dapsone,Acne,"""I have tried almost everything under the sun ...",10,Positive


For 10,000 rows dataframe

In [450]:
b_10k_df =  df.iloc[:10000,:4].copy()
b_10k_df

,drugName,condition,review,rating
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9
...,...,...,...,...
9995,Etonogestrel,Birth Control,"""Where do I begin, I have been on Nexplanon fo...",6
9996,Dienogest / estradiol,Birth Control,"""Natazia made me moody (teary-eyed for no reas...",2
9997,Tioconazole,Vaginal Yeast Infection,"""HORRIBLE, HORRIBLE EXPERIENCE with this drug....",1
9998,Solodyn,Acne,"""I started taking solodyn 80mg for a month aft...",4


In [454]:
reviews_10k = b_10k_df['review'].tolist()

# Example: Get sentiments for all reviews
sentiments = get_sentiments_batch(reviews_10k)
print(f"Number of processed sentiments: {len(sentiments)}")
# Create a new 'sentiment' column in the original dataframe
b_10k_df['sentiment'] = b_10k_df.index.map(sentiments)

100%|██████████| 100/100 [08:54<00:00,  5.35s/it]

Number of processed sentiments: 10000


In [455]:
b_10k_df

,drugName,condition,review,rating,sentiment
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,Neutral
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,Positive
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,Negative
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,Neutral
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,Positive
...,...,...,...,...,...
9995,Etonogestrel,Birth Control,"""Where do I begin, I have been on Nexplanon fo...",6,Neutral
9996,Dienogest / estradiol,Birth Control,"""Natazia made me moody (teary-eyed for no reas...",2,Negative
9997,Tioconazole,Vaginal Yeast Infection,"""HORRIBLE, HORRIBLE EXPERIENCE with this drug....",1,Negative
9998,Solodyn,Acne,"""I started taking solodyn 80mg for a month aft...",4,Negative


For 50,000 rows

In [456]:
b_50k_df =  df.iloc[:50000,:4].copy()
b_50k_df

,drugName,condition,review,rating
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9
...,...,...,...,...
49995,Esomeprazole,Barrett's Esophagus,"""Used Nexium for over 14 years. You need to st...",6
49996,Diprivan,Anesthesia,"""I had my first screening colonoscopy with pro...",10
49997,Magnesium sulfate / potassium sulfate / sodium...,Bowel Preparation,"""THIS is the most vile, disgusting, putrid thi...",10
49998,Abilify,Bipolar Disorde,"""I have been taking Abilify for nearly two yea...",10


In [457]:
reviews_50k = b_50k_df['review'].tolist()

# Example: Get sentiments for all reviews
sentiments = get_sentiments_batch(reviews_50k)
print(f"Number of processed sentiments: {len(sentiments)}")
# Create a new 'sentiment' column in the original dataframe
b_50k_df['sentiment'] = b_50k_df.index.map(sentiments)

100%|██████████| 500/500 [44:45<00:00,  5.37s/it]

Number of processed sentiments: 50000


Scalability Testing

-  Conducted experiments with varying dataset sizes:
-  Initially tested with 1,000 reviews (batch size = 100)
-  Successfully scaled up to process 50,000 reviews (batch size = 1000 in a
single API call)

API Limitation Mitigation
-  The batch processing approach effectively mitigated free API tier limitations
-  Achieved significant efficiency gains by processing thousands of reviews in one
API call

----

Local LLM Experiment:

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Load a LLaMA model for sentiment analysis (replace with an appropriate LLaMA model)
model_name = "meta-llama/Llama-2-7b-chat-hf"  # Example LLaMA model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Initialize the sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Example text for sentiment analysis
example_review = "I really love this product, it's amazing!"
result = sentiment_pipeline(example_review)

# Display the sentiment
print(result)


In [ ]:
from transformers import pipeline

# Load the DistilGPT-2 model using Hugging Face
sentiment_pipeline = pipeline("text-classification", model="distilgpt2")

# Example sentence for sentiment analysis
text = "I love using this product! It has been amazing so far."

# Run sentiment analysis
result = sentiment_pipeline(text)
print(result)


In [7]:
test_df =  df.copy()

In [ ]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
import torch

tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2ForSequenceClassification.from_pretrained('distilgpt2', num_labels=2)

# Add padding token to tokenizer
tokenizer.pad_token = tokenizer.eos_token

In [13]:
def analyze_sentiment_with_gpt(review):
    prompt = f"Classify the sentiment of this review as Positive or Negative:\nReview: {review}\nSentiment: "
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs['input_ids'], max_length=50)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extracting the sentiment from the generated text
    if "Positive" in generated_text:
        return "Positive"
    elif "Negative" in generated_text:
        return "Negative"
    else:
        return "Neutral"


In [ ]:
import pandas as pd

# Apply the GPT-based sentiment analysis function
test_df['sentiment'][:2] = test_df['review'][:2].apply(analyze_sentiment_with_gpt)




In [9]:
def preprocess_text(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    return inputs

# Apply preprocessing to your review column
test_df['processed_reviews'] = test_df['review'][:5].apply(preprocess_text)

In [ ]:
test_df['processed_reviews'][:5]

In [ ]:
def analyze_sentiment(inputs):
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
    return "Positive" if predicted_class == 1 else "Negative"

# Apply sentiment analysis to processed reviews
test_df['sentiment'] = test_df['processed_reviews'].apply(analyze_sentiment)

----

**OpenAI LLM Experiment:** Failed due to limitation of free tier.


In [ ]:
import openai


openai.api_key = "########"

def get_sentiment_openai(review):

    prompt = f"Analyze the sentiment of the following customer review:\n\n\"{review}\"\n\nIs the sentiment Positive, Negative, or Neutral?"

    # Send the prompt to the OpenAI API
    response = openai.ChatCompletion.create(
        model="gpt-4", 
        messages=[
            {"role": "system", "content": "You are a sentiment analysis assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0  # Set to 0 for deterministic results
    )
    
    # Extract and return the sentiment result
    sentiment = response['choices'][0]['message']['content'].strip()
    return sentiment


review = "This drug helped me a lot with my condition, no side effects experienced."
sentiment = get_sentiment_openai(review)
print(f"Review: {review}\nSentiment: {sentiment}")


### **Groq API**

In [458]:
! pip install groq

python(1879) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 1.9 MB/s eta 0:00:00 0:00:01


API key: ************************

In [7]:
import os
print(os.environ["PATH"])

/opt/homebrew/Caskroom/miniconda/base/envs/dsproj/bin:/Users/yatharth/.nvm/versions/node/v22.4.1/bin:/opt/homebrew/Caskroom/miniconda/base/envs/dsproj/bin:/opt/homebrew/Caskroom/miniconda/base/condabin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin


In [8]:
from dotenv import load_dotenv

load_dotenv()

True

Example working of Groq API testing:

In [9]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        # Set an optional system message. This sets the behavior of the
        # assistant and can be used to provide specific instructions for
        # how it should behave throughout the conversation.
        {
            "role": "system",
            "content": "you are a helpful assistant."
        },
        # Set a user message for the assistant to respond to.
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

Fast language models, also known as Efficient Language Models, have gained significant attention in recent years due to their numerous applications and benefits. Here are some of the key reasons why they are important:

1. **Scalability**: Fast language models can process large volumes of text data quickly, making them ideal for applications where speed and efficiency are crucial, such as natural language processing (NLP) and machine learning tasks.
2. **Real-time Processing**: With the ability to process text data in real-time, fast language models can be used in applications like chatbots, voice assistants, and social media analytics, where instant responses are necessary.
3. **Improved Performance**: By leveraging powerful computing architectures and optimized algorithms, fast language models can achieve better performance and accuracy compared to traditional language models, even with limited computational resources.
4. **Cost-Effective**: Fast language models require less computat

Experiment: Initial implementation of Groq API for sentiment analysis on drug reviews.

In [10]:
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

def groq_sentiment(review):

    chat_completion = client.chat.completions.create(
        messages=[
            # Set an optional system message. This sets the behavior of the
            # assistant and can be used to provide specific instructions for
            # how it should behave throughout the conversation.
            {
                "role": "system",
                "content":  "You are customer review sentiment analyzer for medical drugs review. You reply only with Positve, Negative or Neutral."
            },
            
            # Set a user message for the assistant to respond to.
            {
                "role": "user",
                "content": "Give the sentiment of the following review: " + review,
            }
        ],
        model="llama3-8b-8192",
    )

    return chat_completion.choices[0].message.content

# Example usage
review = "This drug helped me a lot with my condition, no side effects experienced."
sentiment = groq_sentiment(review)
print(f"Review: {review}\nSentiment: {sentiment}")


Review: This drug helped me a lot with my condition, no side effects experienced.
Sentiment: Positive


In [11]:
groq_test_reviews = [
    "This drug helped me a lot with my condition, no side effects experienced.",
    "Terrible experience! The medication made my symptoms worse.",
    "No significant change after taking the drug for a month."
]

for r in groq_test_reviews:
    sentiment = groq_sentiment(r)
    print(f"Review: {r}\nSentiment: {sentiment}\n")

Review: This drug helped me a lot with my condition, no side effects experienced.
Sentiment: Positive

Review: Terrible experience! The medication made my symptoms worse.
Sentiment: Negative

Review: No significant change after taking the drug for a month.
Sentiment: Negative



Further, we try to improvise the prompt for sentiment analysis.

In [12]:
import os
from groq import Groq

# Initialize Groq client with API key
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# Function for sentiment analysis of a single review
def groq_sentiment(review):
    # Define the prompt for sentiment analysis
    prompt = f"Analyze the sentiment of the following customer review:\n\n\"{review}\"\n\nIs the sentiment Positive, Negative, or Neutral?" #Improvised prompt

    try:
        # Create chat completion request
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a customer review sentiment analyzer for medical drug reviews. You reply only with Positive, Negative, or Neutral."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            model="llama3-8b-8192",  # Specify model
        )

        # Extract and return the sentiment from the API response
        sentiment = chat_completion.choices[0].message.content.strip()
        return sentiment

    except Exception as e:
        # Handle and log errors
        print(f"Error during sentiment analysis: {e}")
        return "Error"

# Example usage
review = "This drug helped me a lot with my condition, no side effects experienced."
sentiment = groq_sentiment(review)
print(f"Review: {review}\nSentiment: {sentiment}")


Review: This drug helped me a lot with my condition, no side effects experienced.
Sentiment: Positive


In [14]:
for r in groq_test_reviews:
    sentiment = groq_sentiment(r)
    print(f"Review: {r}\nSentiment: {sentiment}\n")

Review: This drug helped me a lot with my condition, no side effects experienced.
Sentiment: Positive

Review: Terrible experience! The medication made my symptoms worse.
Sentiment: Negative

Review: No significant change after taking the drug for a month.
Sentiment: Neutral



In [39]:
import json
import re
# Function for sentiment analysis of a single review with JSON response
def groq_sentiment(review):
    # Define the prompt to instruct the API to return sentiment in JSON format
    prompt = (
        f"Analyze the sentiment of the following customer review and provide the response in JSON format:\n\n\"{review}\"\n\n"
        "The JSON schema should include:\n"
        "{\n"
        "  \"sentiment_analysis\": {\n"
        "    \"sentiment\": \"string (positive, negative, neutral, mixed)\",\n"  #Adding the mixed sentiment for nuanced analysis (fine grained analysis)
        "    \"confidence_score\": \"number (0-1)\"\n"
        "  }\n"
        "}"
    )

    try:
        # Create chat completion request
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a data analyst API capable of sentiment analysis that responds in JSON."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            model="llama3-8b-8192",  # Specify the model
        )

        # Extract the JSON response
        response_content = chat_completion.choices[0].message.content.strip()
        # print(response_content)


        # Use a regular expression to extract the JSON portion
        json_match = re.search(r'\{.*\}', response_content, re.DOTALL)

        if json_match:
            # Extract the JSON part from the response
            json_string = json_match.group(0)

            # Parse the JSON response
            result = json.loads(json_string)
            sentiment = result['sentiment_analysis']['sentiment']
            confidence_score = result['sentiment_analysis']['confidence_score']
            return sentiment, confidence_score
        else:
            print(f"Failed to extract JSON from response. Raw response: {response_content}")
            return "Error", 0.0

    except json.JSONDecodeError as json_error:
        # Handle JSON parsing error
        print(f"Error parsing JSON response: {json_error}")
        print(f"Raw response content: {response_content}")
        return "Error", 0.0

    except Exception as e:
        # Handle other errors
        print(f"Error during sentiment analysis: {e}")
        return "Error", 0.0

# Example usage
review = "This drug helped me a lot with my condition, no side effects experienced."
sentiment, confidence_score = groq_sentiment(review)
print(f"Review: {review}\nSentiment: {sentiment}\nConfidence Score: {confidence_score}")

Review: This drug helped me a lot with my condition, no side effects experienced.
Sentiment: positive
Confidence Score: 0.98


In [40]:
for r in groq_test_reviews:
    sentiment = groq_sentiment(r)
    print(f"Review: {r}\nSentiment: {sentiment}\n")

Review: This drug helped me a lot with my condition, no side effects experienced.
Sentiment: ('positive', 0.9)

Review: Terrible experience! The medication made my symptoms worse.
Sentiment: ('negative', 0.87)

Review: No significant change after taking the drug for a month.
Sentiment: ('negative', 0.85)



In [41]:
groq_df =  df.copy()

In [42]:
groq_df["sentiment"]  = groq_df["review"][:20].apply(groq_sentiment)

In [44]:
import pprint
pd.set_option('display.max_colwidth', 0)
pprint.pprint(groq_df[['review','sentiment']][:20])

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  review  \
0   "It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"                                                                                                                         

Great! The above implementaion of th Groq API is working as expected. And also provided more nuanced and reliable results in termns of more detaield classification and confidence score.

1. Groq API Integration
-  Successfully set up and tested the Groq API using the Llama (llama3-8b-8192)
model for sentiment analysis
- Implemented initial code to analyze individual reviews
2. JSON Output Enforcement
-  Implemented the response_format={"type": "json_object"} parameter to ensure
structured API responses
-  This enhancement improves the consistency and parsability of our sentiment
analysis results
3. Confidence Score Implementation
-  Added confidence score functionality to our sentiment analysis requests
-  This feature provides a measure of certainty for each sentiment classification,
enhancing the reliability of our results

4. Mixed Sentiment Class Addition
-  Following discussions with Rimoun, incorporated a "mixed" sentiment class
- This addition allows for more fine-grained analysis, capturing nuanced opinions
in drug reviews

### Groq API: Batch Processing

In [66]:
import os
import json
from tqdm import tqdm
from groq import Groq

def get_sentiments_batch_groq(reviews, batch_size=10):
    all_sentiments = {}
    
    for i in tqdm(range(0, len(reviews), batch_size)):
        batch = reviews[i:i + batch_size]
        
        # Format the reviews for the batch, numbering each review
        formatted_reviews = "\n".join([f"{j + 1}. {review}" for j, review in enumerate(batch)])
        
        # Define the prompt with clear instructions for JSON response
        prompt = (
            f"Analyze the sentiment of each of the following customer reviews and provide the response in JSON format:\n\n{formatted_reviews}\n\n"
            "The JSON schema should include:\n"
            "{\n"
            "  \"review_number\": \"integer\",\n"
            "  \"sentiment_analysis\": {\n"
            "    \"sentiment\": \"string (positive, negative, neutral)\",\n"
            "    \"confidence_score\": \"number (0-1)\"\n"
            "  }\n"
            "}"
        )

        try:
            # Send batch prompt to Groq API
            chat_completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "system",
                        "content": "You are a customer review sentiment analyzer. Please analyze each review and return the results in JSON format."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                model="llama3-8b-8192" ,
                 # Enforce the response format to be JSON
                response_format={"type": "json_object"} # Specify the model
            )

            # Parse the API response
            response_content = chat_completion.choices[0].message.content.strip()
            # Use a regular expression to extract the JSON portion
            json_match = re.search(r'\{.*\}', response_content, re.DOTALL)
            result = json.loads(json_match.group(0))

            if isinstance(result, list):
                for item in result:
                    absolute_index = i + item['review_number'] - 1
                    all_sentiments[absolute_index] = {
                        'sentiment': item['sentiment_analysis']['sentiment'],
                        'confidence_score': item['sentiment_analysis']['confidence_score']
                    }
            else:
                print(f"Unexpected response format for batch starting at index {i}.")
                print(f"Raw response: {response_content}")

        except json.JSONDecodeError as json_error:
            print(f"Error parsing JSON response for batch starting at index {i}. Raw response content: {response_content}")
            print(f"JSON error: {json_error}")
        except Exception as e:
            print(f"Error during sentiment analysis for batch starting at index {i}: {e}")

    return all_sentiments


In [48]:
# Example: Get sentiments for all reviews
sentiments = get_sentiments_batch_groq(reviews)
print(f"Number of processed sentiments: {len(sentiments)}")

# Create a new 'sentiment' column in the original dataframe
df['sentiment'] = df.index.map(sentiments)

# For debugging, print out any missing reviews
missing_reviews = [idx for idx in range(len(reviews)) if idx not in sentiments]
if missing_reviews:
    print(f"Missing sentiments for the following review indices: {missing_reviews}")

NameError: name 'get_sentiments_batch_groq' is not defined

In [47]:
groq_df_b  = df.iloc[:,:4].copy()
groq_df_b.columns

Index(['drugName', 'condition', 'review', 'rating'], dtype='object')

In [53]:
reviews_1k_groq = groq_df_b['review'][:1000].tolist() 
sentiments = get_sentiments_batch_groq(reviews_1k_groq)
print(f"Number of processed sentiments: {len(sentiments)}")

# Create a new 'sentiment' column in the original dataframe
df['sentiment'] = df.index.map(sentiments)

# For debugging, print out any missing reviews
missing_reviews = [idx for idx in range(len(reviews_1k_groq)) if idx not in sentiments]
if missing_reviews:
    print(f"Missing sentiments for the following review indices: {missing_reviews}")

 10%|█         | 1/10 [00:00<00:06,  1.30it/s]

Error during sentiment analysis for batch starting at index 0: Error code: 400 - {'error': {'message': 'Please reduce the length of the messages or completion.', 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}


 20%|██        | 2/10 [00:00<00:03,  2.26it/s]

Error during sentiment analysis for batch starting at index 100: Error code: 400 - {'error': {'message': 'Please reduce the length of the messages or completion.', 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}


 30%|███       | 3/10 [00:01<00:02,  2.98it/s]

...

 90%|█████████ | 9/10 [00:02<00:00,  3.23it/s]

Error during sentiment analysis for batch starting at index 800: Error code: 400 - {'error': {'message': 'Please reduce the length of the messages or completion.', 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}


100%|██████████| 10/10 [00:03<00:00,  3.20it/s]

Error during sentiment analysis for batch starting at index 900: Error code: 400 - {'error': {'message': 'Please reduce the length of the messages or completion.', 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
Number of processed sentiments: 0


NameError: name 'reviews' is not defined

Received error on running the batch size as 100. As it exceeds the contect length of API.

Now further trying batch zize as 10:

In [68]:
import os
import json
import re
from tqdm import tqdm
from groq import Groq

# Initialize Groq client with API key
# client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# Function for batch sentiment analysis
def get_sentiments_batch_groq(reviews, batch_size=10):
    all_sentiments = {}
    
    # Process the reviews in batches
    for i in tqdm(range(0, len(reviews), batch_size)):
        batch = reviews[i:i + batch_size]
        
        # Formatting reviews with numbering for batch prompt
        formatted_reviews = "\n".join([f"{j + 1}. {review}" for j, review in enumerate(batch)])
        
        # Define the prompt with clear instructions for JSON response
        prompt = (
            f"Analyze the sentiment of each of the following customer reviews and provide the response in JSON format:\n\n{formatted_reviews}\n\n"
            "The JSON schema should include:\n"
            "{\n"
            "  \"review_number\": \"integer\",\n"
            "  \"sentiment_analysis\": {\n"
            "    \"sentiment\": \"string (positive, negative, neutral)\",\n"
            "    \"confidence_score\": \"number (0-1)\"\n"
            "  }\n"
            "}"
        )

        try:
            # Send batch prompt to Groq API
            chat_completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "system",
                        "content": "You are a customer review sentiment analyzer. Please analyze each review and return the results in JSON format."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                model="llama3-8b-8192",
                 # Enforce the response format to be JSON
                response_format={"type": "json_object"}
            )

            # Extract the raw response content
            response_content = chat_completion.choices[0].message.content.strip()

            # Use a regular expression to extract the JSON portion
            json_match = re.search(r'\{.*\}', response_content, re.DOTALL)
            
            if json_match:
                # Extract the JSON part from the response
                json_string = json_match.group(0)

                # Parse the JSON response
                result = json.loads(json_string)

                if isinstance(result, list):
                    for item in result:
                        # Calculate the absolute index in the main list of reviews.
                        absolute_index = i + item['review_number'] - 1  # Adjust for 0-based indexing
                        all_sentiments[absolute_index] = {
                            'sentiment': item['sentiment_analysis']['sentiment'],
                            'confidence_score': item['sentiment_analysis']['confidence_score']
                        }
                else:
                    print(f"Unexpected response format for batch starting at index {i}.")
                    print(f"Raw response: {response_content}")
            else:
                print(f"Failed to extract JSON from response. Raw response: {response_content}")

        except json.JSONDecodeError as json_error:
            print(f"Error parsing JSON response for batch starting at index {i}. Raw response content: {response_content}")
            print(f"JSON error: {json_error}")
        except Exception as e:
            print(f"Error during sentiment analysis for batch starting at index {i}: {e}")

    return all_sentiments

# # Assuming your original dataframe is called 'df' and has a 'review' column
# reviews = df['review'].tolist()

# # Example: Get sentiments for all reviews
# sentiments = get_sentiments_batch_groq(reviews)
# print(f"Number of processed sentiments: {len(sentiments)}")

# # Create a new 'sentiment' column in the original dataframe
# df['sentiment'] = df.index.map(sentiments)

# # For debugging, print out any missing reviews
# missing_reviews = [idx for idx in range(len(reviews)) if idx not in sentiments]
# if missing_reviews:
#     print(f"Missing sentiments for the following review indices: {missing_reviews}")


In [ ]:
reviews_1k_groq = groq_df_b['review'][:1000].tolist() 
sentiments = get_sentiments_batch_groq(reviews_1k_groq)
print(f"Number of processed sentiments: {len(sentiments)}")

# Create a new 'sentiment' column in the original dataframe
df['sentiment'] = df.index.map(sentiments)

# For debugging, print out any missing reviews
missing_reviews = [idx for idx in range(len(reviews_1k_groq)) if idx not in sentiments]
if missing_reviews:
    print(f"Missing sentiments for the following review indices: {missing_reviews}")

In [72]:
import os
import json
from tqdm import tqdm
from groq import Groq

# Initialize Groq client with API key
# client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# Function for batch sentiment analysis
def get_sentiments_batch_groq(reviews, batch_size=5):
    all_sentiments = {}
    
    # Process the reviews in batches
    for i in tqdm(range(0, len(reviews), batch_size)):
        batch = reviews[i:i + batch_size]
        
        # Formatting reviews with numbering for batch prompt
        formatted_reviews = "\n".join([f"{j + 1}. {review}" for j, review in enumerate(batch)])
        
        # Define the prompt with clear instructions for JSON response
        prompt = (
            f"Analyze the sentiment of each of the following customer reviews and provide the response in JSON format:\n\n{formatted_reviews}\n\n"
            "The JSON schema should include:\n"
            "{\n"
            "  \"review_number\": \"integer\",\n"
            "  \"sentiment_analysis\": {\n"
            "    \"sentiment\": \"string (positive, negative, neutral)\",\n"
            "    \"confidence_score\": \"number (0-1)\"\n"
            "  }\n"
            "}"
        )

        try:
            # Send batch prompt to Groq API
            chat_completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "system",
                        "content": "You are a customer review sentiment analyzer. Please analyze each review and return the results in JSON format."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                model="llama3-8b-8192",
                # Enforce the response format to be JSON
                response_format={"type": "json_object"}
            )

            # Extract the raw response content
            response_content = chat_completion.choices[0].message.content.strip()

            # Parse the response, expecting it to be a JSON object with a "reviews" key
            result = json.loads(response_content)

            # Extract the list of reviews from the "reviews" key
            reviews_list = result.get("reviews", [])

            if isinstance(reviews_list, list):
                for item in reviews_list:
                    # Calculate the absolute index in the main list of reviews.
                    absolute_index = i + item['review_number'] - 1  # Adjust for 0-based indexing
                    all_sentiments[absolute_index] = {
                        'sentiment': item['sentiment_analysis']['sentiment'],
                        'confidence_score': item['sentiment_analysis']['confidence_score']
                    }
            else:
                print(f"Unexpected response format for batch starting at index {i}.")
                print(f"Raw response: {response_content}")

        except json.JSONDecodeError as json_error:
            print(f"Error parsing JSON response for batch starting at index {i}. Raw response content: {response_content}")
            print(f"JSON error: {json_error}")
        except Exception as e:
            print(f"Error during sentiment analysis for batch starting at index {i}: {e}")

    return all_sentiments

reviews_1k_groq = groq_df_b['review'][:1000].tolist() 
sentiments = get_sentiments_batch_groq(reviews_1k_groq)
print(f"Number of processed sentiments: {len(sentiments)}")

# Create a new 'sentiment' column in the original dataframe
df['sentiment'] = df.index.map(sentiments)

# For debugging, print out any missing reviews
missing_reviews = [idx for idx in range(len(reviews_1k_groq)) if idx not in sentiments]
if missing_reviews:
    print(f"Missing sentiments for the following review indices: {missing_reviews}")

  0%|          | 1/200 [00:00<01:35,  2.08it/s]

Error during sentiment analysis for batch starting at index 0: Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': '{\n   "review_number": 1,\n   "sentiment_analysis": {\n      "sentiment": "positive",\n      "confidence_score": 0.8\n   }\n}\n{\n   "review_number": 2,\n   "sentiment_analysis": {\n      "sentiment": "positive",\n      "confidence_score": 0.9\n   }\n}\n{\n   "review_number": 3,\n   "sentiment_analysis": {\n      "sentiment": "negative",\n      "confidence_score": 0.7\n   }\n}\n{\n   "review_number": 4,\n   "sentiment_analysis": {\n      "sentiment": "neutral",\n      "confidence_score": 0.5\n   }\n}\n{\n   "review_number": 5,\n   "sentiment_analysis": {\n      "sentiment": "positive",\n      "confidence_score": 0.9\n   }\n}'}}


  2%|▏         | 3/200 [00:06<07:16,  2.21s/it]

Error during sentiment analysis for batch starting at index 10: Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': '{\n   "review_number": 1,\n   "sentiment_analysis": {\n      "sentiment": "positive",\n      "confidence_score": 0.8\n   }\n}\n\n{\n   "review_number": 2,\n   "sentiment_analysis": {\n      "sentiment": "positive",\n      "confidence_score": 0.9\n   }\n}\n\n{\n   "review_number": 3,\n   "sentiment_analysis": {\n      "sentiment": "negative",\n      "confidence_score": 0.7\n   }\n}\n\n{\n   "review_number": 4,\n   "sentiment_analysis": {\n      "sentiment": "negative",\n      "confidence_score": 0.6\n   }\n}\n\n{\n   "review_number": 5,\n   "sentiment_analysis": {\n      "sentiment": "negative",\n      "confidence_score": 0.9\n   }\n}'}}


  3%|▎         | 6/200 [00:14<08:46,  2.71s/it]

...

Error during sentiment analysis for batch starting at index 145: Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': '{\n   "review_number": 1,\n   "sentiment_analysis": {\n      "sentiment": "positive",\n      "confidence_score": 0.9\n   }\n},\n{\n   "review_number": 2,\n   "sentiment_analysis": {\n      "sentiment": "positive",\n      "confidence_score": 0.7\n   }\n},\n{\n   "review_number": 3,\n   "sentiment_analysis": {\n      "sentiment": "positive",\n      "confidence_score": 0.8\n   }\n},\n{\n   "review_number": 4,\n   "sentiment_analysis": {\n      "sentiment": "positive",\n      "confidence_score": 0.9\n   }\n},\n{\n   "review_number": 5,\n   "sentiment_analysis": {\n      "sentiment": "negative",\n      "confidence_score": 0.4\n   }\n}'}}


 16%|█▌        | 31/200 [01:19<07:11,  2.55s/it]

Error during sentiment analysis for batch starting at index 150: Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': '{\n   "review_number":"1",\n   "sentiment_analysis": {\n      "sentiment":"negative",\n      "confidence_score":0.9\n   }\n}\n\n{\n   "review_number":"2",\n   "sentiment_analysis": {\n      "sentiment":"negative",\n      "confidence_score":0.8\n   }\n}\n\n{\n   "review_number":"3",\n   "sentiment_analysis": {\n      "sentiment":"negative",\n      "confidence_score":0.7\n   }\n}\n\n{\n   "review_number":"4",\n   "sentiment_analysis": {\n      "sentiment":"neutral",\n      "confidence_score":0.5\n   }\n}\n\n{\n   "review_number":"5",\n   "sentiment_analysis": {\n      "sentiment":"negative",\n      "confidence_score":0.9\n   }\n}'}}


 16%|█▌        | 31/200 [01:20<07:20,  2.61s/it]


KeyboardInterrupt: 

In [78]:
import os
import json
from tqdm import tqdm
from groq import Groq

# Initialize Groq client with API key
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# Function for batch sentiment analysis
def get_sentiments_batch_groq(reviews, batch_size=10):  # Reduced batch size to avoid token/output size limits
    all_sentiments = {}
    
    # Process the reviews in batches
    for i in tqdm(range(0, len(reviews), batch_size)):
        batch = reviews[i:i + batch_size]
        
        # Formatting reviews with numbering for batch prompt
        formatted_reviews = "\n".join([f"{j + 1}. {review}" for j, review in enumerate(batch)])
        
        # Define the system prompt with clear instructions for JSON response
        system_prompt = (
            "You are a sentiment analyzer for customer reviews. "
            "Analyze the sentiment of each review provided and return the results in valid JSON format. "
            "The JSON should be an array where each object contains 'review_number', 'sentiment', and 'confidence_score'. "
            "Each review should have a sentiment of 'positive', 'negative', or 'neutral', and a confidence score between 0 and 1."
        )

        # Define the user prompt with the list of reviews
        user_prompt = (
            f"Here are the reviews to analyze:\n\n{formatted_reviews}"
        )

        try:
            # Send batch prompt to Groq API
            chat_completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "system",
                        "content": system_prompt  # System message describes the JSON structure
                    },
                    {
                        "role": "user",
                        "content": user_prompt
                    }
                ],
                model="llama3-8b-8192", # Switch to Mixtral, which is better for JSON
                # Enforce the response format to be JSON
                response_format={"type": "json_object"}
            )

            # Extract the raw response content
            response_content = chat_completion.choices[0].message.content.strip()

            # Parse the response, expecting it to be a JSON array
            result = json.loads(response_content)
            # Extract the list of reviews from the "reviews" key
            reviews_list = result.get("reviews", [])

            if isinstance(reviews_list, list):
                for item in reviews_list:
                    # Calculate the absolute index in the main list of reviews.
                    absolute_index = i + item['review_number'] - 1  # Adjust for 0-based indexing
                    all_sentiments[absolute_index] = {
                        'sentiment': item['sentiment_analysis']['sentiment'],
                        'confidence_score': item['sentiment_analysis']['confidence_score']
                    }
            else:
                print(f"Unexpected response format for batch starting at index {i}.")
                print(f"Raw response: {response_content}")

        except json.JSONDecodeError as json_error:
            print(f"Error parsing JSON response for batch starting at index {i}. Raw response content: {response_content}")
            print(f"JSON error: {json_error}")
        except Exception as e:
            print(f"Error during sentiment analysis for batch starting at index {i}: {e}")

    return all_sentiments

reviews_1k_groq = groq_df_b['review'][:1000].tolist() 
sentiments = get_sentiments_batch_groq(reviews_1k_groq)
print(f"Number of processed sentiments: {len(sentiments)}")

# Create a new 'sentiment' column in the original dataframe
df['sentiment'] = df.index.map(sentiments)

# For debugging, print out any missing reviews
missing_reviews = [idx for idx in range(len(reviews_1k_groq)) if idx not in sentiments]
if missing_reviews:
    print(f"Missing sentiments for the following review indices: {missing_reviews}")

  0%|          | 0/100 [00:00<?, ?it/s]

  2%|▏         | 2/100 [00:00<00:41,  2.37it/s]

Error during sentiment analysis for batch starting at index 0: 'sentiment_analysis'
Error during sentiment analysis for batch starting at index 10: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01j9m95282e27t3n1tb7c5d96z` on : Limit 500000, Used 500063, Requested 1704. Please try again in 5m5.5032s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}


  4%|▍         | 4/100 [00:01<00:21,  4.57it/s]

Error during sentiment analysis for batch starting at index 20: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01j9m95282e27t3n1tb7c5d96z` on : Limit 500000, Used 500063, Requested 1565. Please try again in 4m41.365s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
Error during sentiment analysis for batch starting at index 30: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01j9m95282e27t3n1tb7c5d96z` on : Limit 500000, Used 500062, Requested 794. Please try again in 2m28.0192s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}


...

 82%|████████▏ | 82/100 [00:11<00:02,  7.73it/s]

Error during sentiment analysis for batch starting at index 800: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01j9m95282e27t3n1tb7c5d96z` on : Limit 500000, Used 500004, Requested 1382. Please try again in 3m59.5796s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
Error during sentiment analysis for batch starting at index 810: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01j9m95282e27t3n1tb7c5d96z` on : Limit 500000, Used 500003, Requested 1071. Please try again in 3m5.7348s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}


 84%|████████▍ | 84/100 [00:11<00:02,  7.25it/s]

Error during sentiment analysis for batch starting at index 820: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01j9m95282e27t3n1tb7c5d96z` on : Limit 500000, Used 500003, Requested 1086. Please try again in 3m8.2198s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
Error during sentiment analysis for batch starting at index 830: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01j9m95282e27t3n1tb7c5d96z` on : Limit 500000, Used 500002, Requested 1354. Please try again in 3m54.4302s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}


KeyboardInterrupt: 

In [75]:
import os
import json
from tqdm import tqdm
from groq import Groq

# Initialize Groq client with API key
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# Function for batch sentiment analysis
def get_sentiments_batch_groq(reviews, batch_size=3):
    all_sentiments = {}
    
    # Process the reviews in batches
    for i in tqdm(range(0, len(reviews), batch_size)):
        batch = reviews[i:i + batch_size]
        
        # Formatting reviews with numbering for batch prompt
        formatted_reviews = "\n".join([f"{j + 1}. {review}" for j, review in enumerate(batch)])
        
        # Define the prompt with clear instructions for JSON response
        prompt = (
            f"Analyze the sentiment of each of the following customer reviews and return the results as a single JSON array where each review is an object:\n\n{formatted_reviews}\n\n"
            "The JSON array should contain the following structure for each review:\n"
            "[\n"
            "  {\n"
            "    \"review_number\": \"integer\",\n"
            "    \"sentiment_analysis\": {\n"
            "      \"sentiment\": \"string (positive, negative, neutral)\",\n"
            "      \"confidence_score\": \"number (0-1)\"\n"
            "    }\n"
            "  },\n"
            "  ... (one object for each review)\n"
            "]"
        )

        try:
            # Send batch prompt to Groq API
            chat_completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "system",
                        "content": "You are a customer review sentiment analyzer. Please analyze each review and return the results in a single JSON array."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                model="llama3-8b-8192",
                # Enforce the response format to be JSON
                response_format={"type": "json_object"}
            )
 # Extract the raw response content
            response_content = chat_completion.choices[0].message.content.strip()

            # Parse the response, expecting it to be a JSON object with a "reviews" key
            result = json.loads(response_content)

            # Extract the list of reviews from the "reviews" key
            reviews_list = result.get("reviews", [])

            if isinstance(reviews_list, list):
                for item in reviews_list:
                    # Calculate the absolute index in the main list of reviews.
                    absolute_index = i + item['review_number'] - 1  # Adjust for 0-based indexing
                    all_sentiments[absolute_index] = {
                        'sentiment': item['sentiment_analysis']['sentiment'],
                        'confidence_score': item['sentiment_analysis']['confidence_score']
                    }
            else:
                print(f"Unexpected response format for batch starting at index {i}.")
                print(f"Raw response: {response_content}")

        except json.JSONDecodeError as json_error:
            print(f"Error parsing JSON response for batch starting at index {i}. Raw response content: {response_content}")
            print(f"JSON error: {json_error}")
        except Exception as e:
            print(f"Error during sentiment analysis for batch starting at index {i}: {e}")

    return all_sentiments


reviews_1k_groq = groq_df_b['review'][:1000].tolist() 
sentiments = get_sentiments_batch_groq(reviews_1k_groq)
print(f"Number of processed sentiments: {len(sentiments)}")

# Create a new 'sentiment' column in the original dataframe
df['sentiment'] = df.index.map(sentiments)

# For debugging, print out any missing reviews
missing_reviews = [idx for idx in range(len(reviews_1k_groq)) if idx not in sentiments]
if missing_reviews:
    print(f"Missing sentiments for the following review indices: {missing_reviews}")


  3%|▎         | 11/334 [00:09<04:21,  1.24it/s]

Error during sentiment analysis for batch starting at index 30: Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': '{\n   "review_number": 1,\n   "sentiment_analysis": {\n      "sentiment": "positive",\n      "confidence_score": 0.8\n   }\n},\n{\n   "review_number": 2,\n   "sentiment_analysis": {\n      "sentiment": "positive",\n      "confidence_score": 0.6\n   }\n},\n{\n   "review_number": 3,\n   "sentiment_analysis": {\n      "sentiment": "positive",\n      "confidence_score": 0.7\n   }\n}'}}


  4%|▍         | 13/334 [00:10<03:05,  1.73it/s]

Error during sentiment analysis for batch starting at index 36: Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': '{\n   "review_number": 1,\n   "sentiment_analysis": {\n      "sentiment": "negative",\n      "confidence_score": 0.95\n   }\n},\n{\n   "review_number": 2,\n   "sentiment_analysis": {\n      "sentiment": "positive",\n      "confidence_score": 0.85\n   }\n},\n{\n   "review_number": 3,\n   "sentiment_analysis": {\n      "sentiment": "negative",\n      "confidence_score": 0.9\n   }\n}'}}


  5%|▍         | 16/334 [00:11<02:32,  2.08it/s]

...

Error during sentiment analysis for batch starting at index 987: Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': '{\n   "review_number": 1,\n   "sentiment_analysis": {\n      "sentiment": "positive",\n      "confidence_score": 0.90\n   }\n},\n{\n   "review_number": 2,\n   "sentiment_analysis": {\n      "sentiment": "positive",\n      "confidence_score": 0.70\n   }\n},\n{\n   "review_number": 3,\n   "sentiment_analysis": {\n      "sentiment": "positive",\n      "confidence_score": 0.80\n   }\n}'}}
Error during sentiment analysis for batch starting at index 990: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01j9m95282e27t3n1tb7c5d96z` on : Limit 500000, Used 499749, Requested 603. Please try again in 1m0.7506s. Visit https://console.groq.com/docs/rate-lim

100%|█████████▉| 333/334 [07:59<00:11, 11.18s/it]

Error during sentiment analysis for batch starting at index 996: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01j9m95282e27t3n1tb7c5d96z` on : Limit 500000, Used 500114, Requested 411. Please try again in 1m30.7678s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}


100%|██████████| 334/334 [08:58<00:00,  1.61s/it]

Number of processed sentiments: 153
Missing sentiments for the following review indices: [9, 10, 11, 12, 13, 14, 18, 19, 20, 21, 22, 23, 30, 31, 32, 33, 34, 35, 36, 37, 38, 45, 46, 47, 48, 49, 50, 51, 52, 53, 60, 61, 62, 63, 64, 65, 66, 67, 68, 75, 76, 77, 84, 85, 86, 93, 94, 95, 105, 106, 107, 108, 109, 110, 114, 115, 116, 117, 118, 119, 132, 133, 134, 135, 136, 137, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 159, 160, 161, 162, 163, 164, 171, 172, 173, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 192, 193, 194, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 219, 220, 221, 228, 229, 230, 231, 232, 233, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 258, 259, 260, 261, 262, 263, 264, 265, 266, 273, 274, 275, 279, 280, 281, 285, 286, 287, 288, 289, 290, 294, 295, 296, 303, 304, 305, 306, 307, 308, 309, 310, 311, 315, 316, 317, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338,

Context length of the Groq API is small allows for very small batch size, and the need for constraint review size as well.

mistral, aws comprehend , amazon bedrock